## E10-1
#### This Notebook illustrates the use of "MAP-REDUCE" to calculate averages from the data contained in nsedata.csv.

### <b>Task 1</b>
You are required to review the code (refer to the SPARK document where necessary), and <b>add comments / markup explaining the code in each cell</b>. Also explain the role of each cell in the overall context of the solution to the problem (ie. what is the cell trying to achieve in the overall scheme of things). You may create additional code in each cell to generate any debug output that you may need to complete this exercise.
### <b>Task 2</b>
You are required to write code to solve the problem stated at the end this Notebook
### <b>Submission</b>
Create and upload a PDF of this Notebook. <b> BEFORE CONVERTING TO PDF and UPLOADING ENSURE THAT YOU REMOVE / TRIM LENGTHY DEBUG OUTPUTS </b>. Short debug outputs of up to 5 lines are acceptable.


In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql.types import *

In [ ]:
sc = pyspark.SparkContext(appName="E10")

In [ ]:
rdd1 = sc.textFile("/home/hduser/spark/nsedata.csv")

In [ ]:
rdd1 = rdd1.filter(lambda x: "SYMBOL" not in x)

In [ ]:
rdd2 = rdd1.map(lambda x : x.split(","))

In [ ]:
# Helper comment!: The goal is to find out the mean of the OPEN prices and the mean of the CLOSE price in one batch of tasks ...

In [ ]:
rdd_open = rdd2.map(lambda x : (x[0]+"_open",float(x[2])))
rdd_close = rdd2.map(lambda x : (x[0]+"_close",float(x[5])))

In [ ]:
rdd_united = rdd_open.union(rdd_close)

In [ ]:
reducedByKey = rdd_united.reduceByKey(lambda x,y: x+y)

In [ ]:
temp1 = rdd_united.map(lambda x: (x[0],1)).countByKey()
countOfEachSymbol = sc.parallelize(temp1.items())

In [ ]:
symbol_sum_count = reducedByKey.join(countOfEachSymbol)

In [ ]:
averages = symbol_sum_count.map(lambda x : (x[0], x[1][0]/x[1][1]))

In [ ]:
averagesSorted = averages.sortByKey()

In [ ]:
averagesSorted.saveAsTextFile("/home/hduser/spark/averages")

In [ ]:
ss.stop()

### Review the output files generated in the above step and copy the first 15 lines of any one of the output files into the cell below for reference. Write your comments on the generated output

## Task 2 - Problem Statement
### Using the MAP-REDUCE strategy, write SPARK code that will create the average of HIGH prices for all the traded companies, but only for any 3 months of your choice. Create the appropriate (K,V) pairs so that the averages are simultaneously calculated, as in the above example. Create the output files such that the final data is sorted in <b>descending order</b> of the company names.